In [39]:

# coding: utf-8

# In[ ]:


from __future__ import division


# In[ ]:


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
import cv2
import os
from skimage import io as ski_io
from skimage.viewer import ImageViewer
from skimage import color as ski_color
from skimage import io as ski_io
import time
from nolearn.lasagne import BatchIterator
from skimage import transform as ski_transform
NUM_CLASS=13
def one_hot(labels, n_class = NUM_CLASS):
    total_record = len(labels)
    y = np.zeros((total_record, n_class))
    y[np.arange(total_record), labels] = 1
    return y


from collections import namedtuple

Parameters = namedtuple('Parameters', [
        # Data parameters
        'num_classes', 'image_size', 
        # Training parameters
        'batch_size', 'max_epochs', 'log_epoch', 'print_epoch',
        # Optimisations
        'learning_rate_decay', 'learning_rate',
        'l2_reg_enabled', 'l2_lambda', 
        'early_stopping_enabled', 'early_stopping_patience', 
        'resume_training', 
        # Layers architecture
        'conv1_k', 'conv1_d', 'conv1_p', 
        'conv2_k', 'conv2_d', 'conv2_p', 
        'conv3_k', 'conv3_d', 'conv3_p', 
        'fc4_size', 'fc4_p'
    ])

import os

class Paths(object):
    """
    Provides easy access to common paths we use for persisting 
    the data associated with model training.
    """
    
    def __init__(self, params):
        """
        Initialises a new `Paths` instance and creates corresponding folders if needed.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
        """
        self.model_name = self.get_model_name(params)
        self.var_scope = self.get_variables_scope(params)
        self.root_path = os.getcwd() + "/models/" + self.model_name + "/"
        self.model_path = self.get_model_path()
        self.train_history_path = self.get_train_history_path()
        self.learning_curves_path = self.get_learning_curves_path()
        if not os.path.exists(self.root_path):
            os.makedirs(self.root_path)

    def get_model_name(self, params):
        """
        Generates a model name with some of the crucial model parameters encoded into the name.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Model name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        print ("call get model name")
        model_name = "k{}d{}p{}_k{}d{}p{}_k{}d{}p{}_fc{}p{}".format(
            params.conv1_k, params.conv1_d, params.conv1_p, 
            params.conv2_k, params.conv2_d, params.conv2_p, 
            params.conv3_k, params.conv3_d, params.conv3_p, 
            params.fc4_size, params.fc4_p
        )
        model_name += "_lrdec" if params.learning_rate_decay else "_no-lrdec"
        model_name += "_l2" if params.l2_reg_enabled else "_no-l2"
        return model_name

    def get_variables_scope(self, params):
        """
        Generates a model variable scope with some of the crucial model parameters encoded.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Variables scope name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        var_scope = "k{}d{}_k{}d{}_k{}d{}_fc{}_fc0".format(
            params.conv1_k, params.conv1_d,
            params.conv2_k, params.conv2_d,
            params.conv3_k, params.conv3_d, 
            params.fc4_size
        )
        return var_scope

    def get_model_path(self):
        """
        Generates path to the model file.
   
        Returns
        -------
        Model file path.
        """
        return self.root_path + "model.ckpt"

    def get_train_history_path(self):
        """
        Generates path to the train history file.
   
        Returns
        -------
        Train history file path.
        """
        return self.root_path + "train_history"
    
    def get_learning_curves_path(self):
        """
        Generates path to the learning curves graph file.
   
        Returns
        -------
        Learning curves file path.
        """
        return self.root_path + "learning_curves.png"


# In[ ]:


def fully_connected(input, size):
    """
    Performs a single fully connected layer pass, e.g. returns `input * weights + bias`.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer = tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    return tf.nn.relu(fully_connected(input, size))

def conv_relu(input, kernel_size, depth):
    """
    Performs a single convolution layer pass.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer = tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides = [1, 1, 1, 1], padding = 'SAME')
    return tf.nn.relu(conv + biases)

def pool(input, size):
    """
    Performs a max pooling layer pass.
    """
    return tf.nn.max_pool(
        input, 
        ksize = [1, size, size, 1], 
        strides = [1, size, size, 1], 
        padding = 'SAME'
    )

def model_pass(input, params, is_training):
    # Convolutions

    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = params.conv1_k, depth = params.conv1_d) 
    with tf.variable_scope('pool1'): 
        pool1 = pool(conv1, size = 2)
        pool1 = tf.cond(is_training, lambda: tf.nn.dropout(pool1, keep_prob = params.conv1_p), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = params.conv2_k, depth = params.conv2_d)
    with tf.variable_scope('pool2'):
        pool2 = pool(conv2, size = 2)
        pool2 = tf.cond(is_training, lambda: tf.nn.dropout(pool2, keep_prob = params.conv2_p), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = params.conv3_k, depth = params.conv3_d)
    with tf.variable_scope('pool3'):
        pool3 = pool(conv3, size = 2)
        pool3 = tf.cond(is_training, lambda: tf.nn.dropout(pool3, keep_prob = params.conv3_p), lambda: pool3)
    
    # Fully connected
    
    # 1st stage output
    pool1 = pool(pool1, size = 4)
    shape = pool1.get_shape().as_list()
    pool1 = tf.reshape(pool1, [-1, shape[1] * shape[2] * shape[3]])
    
    # 2nd stage output
    pool2 = pool(pool2, size = 2)
    shape = pool2.get_shape().as_list()
    pool2 = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])    
    
    # 3rd stage output
    shape = pool3.get_shape().as_list()
    pool3 = tf.reshape(pool3, [-1, shape[1] * shape[2] * shape[3]])
    
    flattened = tf.concat([pool1, pool2, pool3], 1)
    
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = params.fc4_size)
        fc4 = tf.cond(is_training, lambda: tf.nn.dropout(fc4, keep_prob = params.fc4_p), lambda: fc4)
    with tf.variable_scope('out'):
        logits = fully_connected(fc4, size = params.num_classes)
    return logits


parameters = Parameters(
    # Data parameters
    num_classes = NUM_CLASS,
    image_size = (32, 32),
    # Training parameters
    batch_size = 256,
    max_epochs = 100,
    log_epoch = 1,
    print_epoch = 1,
    # Optimisations
    learning_rate_decay = False,
    learning_rate = 0.0001,
    l2_reg_enabled = True,
    l2_lambda = 0.0001,
    early_stopping_enabled = True,
    early_stopping_patience = 100,
    resume_training = True,
    # Layers architecture
    conv1_k = 5, conv1_d = 32, conv1_p = 0.9,
    conv2_k = 5, conv2_d = 64, conv2_p = 0.8,
    conv3_k = 5, conv3_d = 128, conv3_p = 0.7,
    fc4_size = 1024, fc4_p = 0.5
)

In [40]:
import cv2
import numpy as np

In [41]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [42]:
map_class = ['stop', 'turn_left', 'turn_right', 'prob_left', 'prob_right', 'no_entry', 'speed_limit', 'min_speed', 'end_min', 'unknown', 'left', 'right', 'other']
map_answer = [1,2,3,4,5,6,7,8,8,8,8,8,13]

In [43]:
# threshold
top_thres=40
bottom_thres=50
prev_check_thres=5
prev_w_thres=5
prev_h_thres=5
roi_id=0
rand_appear_w_thres=20
rand_appear_h_thres=20


In [44]:
import cv2
import numpy as np
cap = cv2.VideoCapture('./input-sample/MVI_1063.avi')
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rois = np.zeros([num_frames, 5, 5]) #num frame, max roi per frame, properties
sign_cnt = np.zeros([num_frames * 5])

# preperty

In [45]:
def backtrack(frame_id, x, y, w, h, depth):
    if depth >= prev_check_thres:
        return True
    if frame_id < 0:
        return False
    for frame_p in frame_rois[frame_id-1]:
        None

In [46]:
# Initialisation routines: generate variable scope, create logger, note start time.
paths = Paths(parameters)

# Build the graph
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_x = tf.placeholder(tf.float32, shape = (None, parameters.image_size[0], parameters.image_size[1], 1))
    is_training = tf.constant(False)
    with tf.variable_scope(paths.var_scope):
        predictions = tf.nn.softmax(model_pass(tf_x, parameters, is_training))
        top_k_predictions = tf.nn.top_k(predictions, 5)

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    tf.train.Saver().restore(session, paths.model_path)        
    while (1):

        _, frame = cap.read()
        frame = cv2.resize(frame, (640, 360), interpolation = cv2.INTER_LINEAR)
        blur = cv2.medianBlur(frame.copy(), 5)

        # Convert BGR to HSV
        hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        # define range of blue color in HSV
        lower_blue = np.array([100, 70, 40])
        upper_blue = np.array([130, 255, 255])

        lower_red = np.array([0, 70, 50])
        upper_red = np.array([10, 255, 255]) 
        r_mask1 = cv2.inRange(hsv, lower_red, upper_red) 
        r_mask2 = cv2.inRange(hsv, np.array([100, 10, 170]), np.array([180, 230, 255])) 
        r_mask3 = cv2.inRange(hsv, np.array([157, 60, 11]), np.array([180, 255, 255])) 
        red_mask = r_mask1 | r_mask2 | r_mask3
        

        # Threshold the HSV image to get only blue colors
        blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
        mask = blue_mask + red_mask

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(frame, frame, mask=mask)
        mask = cv2.erode(mask.copy(), None, iterations=2)
        mask2 = cv2.dilate(mask.copy(), None, iterations=2)

        cnts = cv2.findContours(mask2.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)[-2]
        framedraw = frame.copy()
#         framedraw = cv2.cvtColor(framedraw, cv2.COLOR_BGR2GRAY)
#         framedraw = cv2.Canny(framedraw, 100, 200)
        if len(cnts) > 0:
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
#                 Vi tri cac bien            
                if w < 20 or h < 20 or y < 30 or h / w > 2 or w / h > 1.5:
                    cv2.rectangle(framedraw, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    continue
                roi = frame[y:y+h, x:x+w]
                print (y, x, y +h, x + w)
                # extract image and feed to cnn
                roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)  
                roi = ski_transform.resize(roi, (32, 32))
                roi = ski_color.rgb2gray(roi)
                roi = roi.reshape(roi.shape + (1,))        
                tf_X = np.array([roi])
                cv2.rectangle(framedraw, (x, y), (x + w, y + h), (255,0,0), 2)
                [prediction] = session.run([top_k_predictions], feed_dict = {
                        tf_x : tf_X
                    }
                )
                prob_class =prediction[0][0][0] 
                class_ord = prediction[1][0][0]
                class_name = map_class[class_ord]
                cv2.putText(framedraw, class_name + "-" + str(prob_class), (x-100, y + int(h / 2)), cv2.FONT_HERSHEY_SIMPLEX, 1,color=(0,0,255))
        cv2.imshow('frame', framedraw)

        k = cv2.waitKey(10) & 0xFF
        if k == 27:
            break

cv2.destroyAllWindows()

call get model name
INFO:tensorflow:Restoring parameters from /home/tuyenlv17/data/programming/projects/digital-race-2/GSS-CNN-classification/models/k5d32p0.9_k5d64p0.8_k5d128p0.7_fc1024p0.5_no-lrdec_l2/model.ckpt
138 16 327 269
138 15 327 269
96 143 118 164
138 16 327 269
136 16 327 269


/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


137 13 330 269
138 8 332 269
136 8 331 269
136 15 329 270
137 7 328 270
96 148 118 168
137 12 326 271
135 5 327 271
95 146 115 166
135 13 327 272
135 17 328 272
95 146 118 166
136 15 330 272
136 16 331 272
136 18 331 272
137 17 332 272
136 16 333 272
136 20 332 272
136 19 332 272
96 144 118 164
136 18 332 272
135 20 333 272
96 143 116 164
134 20 332 272
96 144 116 164
134 19 330 273
95 143 116 164
134 19 330 273
94 141 114 164
132 18 326 273
132 17 324 273
93 144 117 164
128 17 319 273
91 144 113 164
123 16 318 274
90 142 112 164
122 14 318 274
90 144 111 164
127 13 317 275
89 145 110 165
121 12 316 274
123 13 316 274
123 16 313 274
87 143 108 164
119 75 269 274
118 73 271 274
85 145 106 165
121 72 268 274
84 144 108 165
116 71 268 275
115 70 267 275
118 68 267 275
114 67 267 275
80 142 101 162
113 66 268 275
113 65 268 275
116 63 268 275
78 140 100 160
118 62 268 275
117 60 269 275
117 60 269 275
76 142 99 162
116 59 270 275
77 140 98 161
116 57 270 275
75 140 96 160
116 55 272 275
11

159 199 203 249
146 32 291 211
220 134 283 193
160 197 204 250
147 28 294 210
263 155 288 175
239 150 263 172
160 197 205 249
147 25 292 210
264 151 292 173
241 128 287 169
160 195 206 248
147 27 285 209
242 125 289 168
160 192 207 247
147 24 287 209
247 143 268 163
250 138 272 159
162 188 210 244
146 21 285 205
162 185 212 244
148 20 282 203
191 19 261 120
163 183 213 243
148 68 275 202
164 179 216 239
240 27 266 59
166 175 221 236
151 63 274 196
243 24 268 50
152 62 277 235
245 23 269 52
151 60 280 234
170 167 227 233
154 61 275 193
155 57 277 231
171 165 229 230
155 55 257 191
173 161 234 227
157 64 263 188
173 161 235 227
157 64 267 187
157 65 266 227
155 62 268 224
216 59 236 83
176 154 239 222
156 62 266 183
176 150 242 222
157 55 265 182
219 52 239 78
178 154 233 220
157 56 268 180
178 149 234 216
157 49 271 178
158 48 274 214
159 44 276 213
195 139 248 211
162 42 278 197
163 36 282 208
161 34 284 205
162 32 283 193
163 29 285 184
162 30 284 188
166 24 287 184
165 27 290 185
169

243 479 282 514
206 323 246 353
90 457 181 550
82 18 105 38
208 322 247 353
91 459 183 554
82 14 104 34
241 483 289 518
209 321 249 352
91 462 183 558
81 10 104 31
243 485 292 522
213 316 238 348
92 464 185 562
81 6 104 27
243 487 294 525
222 326 253 350
92 466 187 565
82 0 104 23
258 490 299 528
92 468 189 568
263 493 303 533
219 316 258 350
92 470 190 572
267 494 307 536
227 321 260 347
94 472 193 576
254 496 312 540
221 312 248 344
94 474 196 580
257 498 315 545
223 312 262 344
95 476 198 584
277 502 319 545
224 307 265 344
95 478 200 589
262 505 324 551
226 307 267 343
95 482 203 594
266 508 327 556
227 303 269 346
95 484 205 598
269 511 335 559
229 302 272 344
96 487 209 604
288 514 338 567
230 299 274 342
97 491 213 610
292 518 344 568
231 298 276 339
99 495 216 616
297 522 350 575
232 295 280 343
101 499 221 622
301 526 355 580
234 294 282 341
102 502 250 628
235 292 285 342
171 253 192 273
102 506 269 634
309 535 360 590
238 291 288 341
172 251 193 271
102 510 273 640
309 553 3

38 422 65 453
40 422 64 449
40 422 64 449
40 423 63 451
40 424 63 452
39 422 65 452
38 423 64 452
37 422 65 452
38 333 61 363
41 424 70 456
41 426 69 457
41 425 70 456
39 425 67 455
38 425 67 454
38 427 63 454
71 323 103 353
38 428 64 454
39 429 64 455
73 322 104 354
41 429 63 455
41 466 71 488
42 468 73 491
43 470 75 493
43 470 74 494
42 471 75 497
56 442 79 463
44 472 76 492
44 465 79 494
44 472 77 497
45 475 78 499
45 468 81 501
45 477 78 504
46 476 79 506
44 472 82 509
45 474 82 511
45 474 81 512
45 475 83 514
46 477 83 516
47 479 83 519
47 480 82 520
47 481 82 522
46 482 82 523
47 483 82 524
47 485 82 527
49 490 81 530
49 489 83 531
49 492 83 533
48 492 83 533
48 489 83 534
47 490 81 535
45 493 80 536
45 494 80 537
45 494 80 538
44 491 79 539
43 493 79 541
42 494 78 541
42 496 77 542
41 494 77 543
41 496 76 544
54 584 74 607
54 586 74 610
105 393 158 446
105 395 160 448
107 398 161 452
109 399 163 454
110 400 165 456
109 401 166 457
111 402 166 458
111 403 168 460
111 404 168 461
